In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# training script
script_dir = prefix.joinpath("code", "lightgbm", "iris")
script_name = "train.py"

# environment file
environment_file = prefix.joinpath("environments", "lightgbm.txt")

# azure ml settings
environment_name = "hpo-lightgbm"
experiment_name = "hpo-lightgbm-hyperdrive-loss"
compute_target = "cpu-cluster"

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment

ds = ws.datasets["iris"]

env = Environment.from_pip_requirements(environment_name, environment_file)

arguments = ["--data-dir", ds.as_mount()]

src = ScriptRunConfig(
    source_directory=script_dir,
    script=script_name,
    arguments=arguments,
    environment=env,
    compute_target=compute_target,
)

# run = Experiment(ws, experiment_name).submit(src)
# run

In [ ]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import (
    RandomParameterSampling,
    BayesianParameterSampling,
)
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import (
    choice,
    loguniform,
    uniform,
)

param_sampling = RandomParameterSampling(
    {
        "--num-boost-round": choice(range(1, 100)),
        "--boosting": choice(["gbdt", "rf", "dart", "goss"]),
        "--num-iterations": choice(range(1, 100)),
        "--num-leaves": choice(range(2, 64)),
        "--num-threads": choice([1, 2, 4]),
        "--learning-rate": loguniform(10e-5, 0.1),
        "--seed": choice([1, 3, 5, 7, 11, 13, 42]),
    }
)

hdc = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="loss",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=200,
    max_concurrent_runs=20,
)

run = Experiment(ws, experiment_name).submit(hdc)
run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
%%time

run.wait_for_completion()
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()["runDefinition"]["arguments"]

In [ ]:
best_run.get_metrics()